In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain.tools import tool, ToolRuntime
from langchain.chat_models import init_chat_model

MODEL_NAME = "gemini-2.5-flash-lite"

model = init_chat_model(model=MODEL_NAME, model_provider="google_genai")

@tool
def read_email(runtime: ToolRuntime) -> str:
    """Read an email from the given address."""
    # take email from state
    return runtime.state["email"]

@tool
def send_email(body: str) -> str:
    """Send an email to the given address with the given subject and body."""
    # fake email sending
    return f"Email sent"

In [ ]:
from langchain.agents import create_agent, AgentState
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware

class EmailState(AgentState):
    email: str

agent = create_agent(
    model=model,
    tools=[read_email, send_email],
    state_schema=EmailState,
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "read_email": False,
                "send_email": True,
            },
            description_prefix="Tool execution requires approval",
        ),
    ],
    system_prompt=(
        "You are an email assistant. You MUST follow these steps exactly:\n"
        "1. Use the read_email tool to read the email.\n"
        "2. Immediately use the send_email tool to send a professional reply. "
        "Do NOT ask the user any questions. Draft the reply yourself and send it right away."
    ),
)

In [ ]:
from langchain.messages import HumanMessage

initial_input = {
    "messages": [HumanMessage(content="Please read my email and send a response immediately. Send the reply now in the same thread.")],
    "email": "Hi Seán, I'm going to be late for our meeting tomorrow. Can we reschedule? Best, John."
}

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(initial_input, config=config)

In [ ]:
from pprint import pprint

pprint(response)

In [ ]:
print(response.get('__interrupt__', 'No interrupt — model did not call send_email'))

In [ ]:
# Access just the 'body' argument from the tool call
if '__interrupt__' in response:
    print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])
else:
    print("No interrupt to inspect")

## Approve

In [ ]:
from langgraph.types import Command

approve_config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    Command( 
        resume={"decisions": [{"type": "approve"}]}
    ), 
    config=approve_config
)

pprint(response)

## Reject

In [ ]:
reject_config = {"configurable": {"thread_id": "2"}}

# Start a fresh conversation on a new thread
response = agent.invoke(initial_input, config=reject_config)

# Reject the drafted email with feedback
response = agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "reject",
                    "message": "No please sign off - Your merciful leader, Seán."
                }
            ]
        }
    ), 
    config=reject_config
)   

pprint(response)

In [ ]:
if '__interrupt__' in response:
    print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])
else:
    print(response['messages'][-1].content)

## Edit

In [ ]:
edit_config = {"configurable": {"thread_id": "3"}}

# Start a fresh conversation on a new thread
response = agent.invoke(initial_input, config=edit_config)

# Edit the drafted email with a custom body
response = agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "edit",
                    "edited_action": {
                        "name": "send_email",
                        "args": {"body": "This is the last straw, you're fired!"},
                    }
                }
            ]
        }
    ), 
    config=edit_config
)   

pprint(response)